# Um breve tour do Fatiando a Terra

In [ ]:
import fatiando

In [ ]:
print(fatiando.__version__)

## Criando formas geométricas

In [ ]:
from fatiando import mesher

In [ ]:
prisma = mesher.Prism(1, 5, 9, 29, 0, 3)

In [ ]:
prisma

In [ ]:
prisma.get_bounds()

In [ ]:
prisma.addprop('density', 250)

In [ ]:
prisma.props

In [ ]:
from fatiando.vis import myv

In [ ]:
myv.figure()
myv.prisms([prisma])
myv.show()

In [ ]:
myv.figure()
myv.prisms([prisma], 'density')
bounds = [0, 20, 0, 30, 0, 20]
myv.outline(bounds)
myv.wall_north(bounds)
myv.show()

## Desafio: Crie e visualize um modelo de 5 prismas

In [ ]:
modelo = [prisma, mesher.Prism(10, 15, 2, 5, 5, 10, {'density': -250})]

In [ ]:
myv.figure()
myv.prisms(modelo, 'density')
bounds = [0, 20, 0, 30, 0, 20]
myv.outline(bounds)
myv.wall_north(bounds)
myv.show()

## Modelagem direta

In [ ]:
from fatiando import gridder

In [ ]:
x, y = gridder.regular(bounds[:4], (50, 50))

In [ ]:
x, y

In [ ]:
import numpy as np

In [ ]:
z = np.zeros_like(x)

In [ ]:
z

In [ ]:
from fatiando.gravmag import prism

In [ ]:
grav = prism.gz(x, y, z, modelo)

In [ ]:
grav

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.tricontourf(y, x, grav, 60)

## Desafio: modelo de prismas da topografia

In [ ]:
x, y, topo = np.loadtxt('topografia.txt', unpack=True)

In [ ]:
dx = (x.max() - x.min())/(20 - 1)
dy = (y.max() - y.min())/(20 - 1)

In [ ]:
dx, dy

In [ ]:
modelo = []
for i in xrange(len(x)):
    prisma = mesher.Prism(x[i] - dx/2, x[i] + dx/2,
                          y[i] - dy/2, y[i] + dy/2,
                          0, -topo[i], {'density': 2670})
    modelo.append(prisma)

In [ ]:
myv.figure()
myv.prisms(modelo)
myv.show()

## Dados grav

In [ ]:
lon, lat, y, x, alt, topo, grav = np.loadtxt('gravidade-havai.txt', unpack=True)

In [ ]:
plt.tricontourf(y, x, grav, 60)

In [ ]:
from fatiando.gravmag import normal_gravity

In [ ]:
disturbio = grav - normal_gravity.gamma_closed_form(lat, alt)

In [ ]:
plt.tricontourf(y, x, disturbio, 60)
plt.colorbar()

In [ ]:
bouguer = disturbio - normal_gravity.bouguer_plate(topo)

In [ ]:
plt.tricontourf(y, x, bouguer, 60)
plt.colorbar()

## Desafio: calcule a correção topográfica

In [ ]:
dx, dy = (x.max() - x.min())/100, (y.max() - y.min())/100


In [ ]:
modelo = []
for i in xrange(len(x)):
    if topo[i] < 0:
        prisma = mesher.Prism(x[i] - dx/2, x[i] + dx/2,
                              y[i] - dy/2, y[i] + dy/2,
                              0, -topo[i])
        prisma.addprop('density', -1630)
    else:
        prisma = mesher.Prism(x[i] - dx/2, x[i] + dx/2,
                              y[i] - dy/2, y[i] + dy/2,
                              -topo[i], 0)
        prisma.addprop('density', 2670)
    modelo.append(prisma)    

In [ ]:
def gera_modelo(x, y, topo, dx, dy):
    for i in xrange(len(x)):
        if topo[i] < 0:
            z1, z2, dens = 0, -topo[i], -1630
        else:
            z1, z2, dens = -topo[i], 0, 2670
        prisma = mesher.Prism(x[i] - dx/2, x[i] + dx/2,
                              y[i] - dy/2, y[i] + dy/2,
                              z1, z2, {'density':dens})
        yield prisma    

In [ ]:
modelo = gera_modelo(x, y, topo, dx, dy)

In [ ]:
myv.figure()
myv.prisms(modelo, 'density', scale=(1, 1, 50))
myv.show()

In [ ]:
modelo = gera_modelo(x, y, topo, dx, dy)

In [ ]:
%time grav_topo = prism.gz(x, y, -alt, modelo)

In [ ]:
bouguer2 = disturbio - grav_topo

In [ ]:
plt.tricontourf(y, x, bouguer2, 60)
plt.colorbar()